## Introduction

Lorem ipsum, setting up the entire project for 2 laptops...

**Project description**: lorem ipsum: 

High level steps (table of content, internal links) goes here: 

On [this link (Kaggle)](https://www.kaggle.com/c/titanic/data) you can find the original dataset and its description.

*Project owner*: ***Gustaf Olofsson***

## Load Data from CSVs

In [ ]:
import unicodecsv

#won't need this, use pandas built-in function
def read_csv(filename):
    with open(filename, 'rb') as f:
        reader = unicodecsv.DictReader(f)
        return list(reader)


## SECTION TITLE

## SECTION TITLE

## SECTION TITLE

## SECTION TITLE

## SECTION TITLE